# Selenium Webscraping of Fidelity Stock Research
More information on how the notebook works will be added later.

In [1]:
# imports
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
import pandas as pd